In [19]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
import logging

# Configure file paths via environment variables or defaults
BASE_DIR = Path(os.environ.get('MC_BASE_DIR', r'E:\\Automation\\Card_recon\\original reports'))
LOE_FILE = Path(os.environ.get('MC_LOE_FILE', 'LOE20250219.xlsx'))
OUTGOING_FILE = Path(os.environ.get('MC_OUT_FILE', 'All Outgoing Transaction Details TWI_2025_02_20_080441.xlsx'))
LOE_PATH = BASE_DIR / LOE_FILE
OUTGOING_PATH = BASE_DIR / OUTGOING_FILE

# Ensure report output directory exists (defaults to mc_reports inside this repo)
REPORT_DIR = Path(os.environ.get('MC_REPORTS_DIR', r'E:\\Automation\\Card_recon\\MC_\\mc_reports'))
REPORT_DIR.mkdir(parents=True, exist_ok=True)

# Configure log file (defaults to conversion.log in notebook working directory)
LOG_FILE = Path(os.environ.get('MC_LOG_FILE', str(Path.cwd() / 'conversion.log')))

# Setup a dedicated logger
logger = logging.getLogger('mc_recon')
if not logger.handlers:
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler(str(LOG_FILE), mode='a', encoding='utf-8')
    fmt = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    fh.setFormatter(fmt)
    logger.addHandler(fh)

logger.info(f'Configured paths: LOE_PATH={LOE_PATH} OUTGOING_PATH={OUTGOING_PATH} REPORT_DIR={REPORT_DIR} LOG_FILE={LOG_FILE}')
print('Configured paths:')
print(f' LOE_PATH={LOE_PATH}')
print(f' OUTGOING_PATH={OUTGOING_PATH}')
print(f' REPORT_DIR={REPORT_DIR}')

Configured paths:
 LOE_PATH=E:\Automation\Card_recon\original reports\LOE20250219.xlsx
 OUTGOING_PATH=E:\Automation\Card_recon\original reports\All Outgoing Transaction Details TWI_2025_02_20_080441.xlsx
 REPORT_DIR=E:\Automation\Card_recon\MC_\mc_reports


In [20]:
print("Processing Files started. This might take a minute... PLEASE WAIT.")
def _normalize_cols(cols):
    return [str(c).strip() for c in cols]

def _col_key(c):
    return str(c).lower().replace(' ', '').replace(' ','')

def detect_and_read_loe(path, expected_cols, engine='openpyxl', max_header=4):
    try:
        sheets = pd.read_excel(str(path), sheet_name=None, engine=engine)
    except Exception as e:
        logger.exception(f'Failed to open LOE workbook: {e}')
        raise
    expected_keys = [ _col_key(c) for c in expected_cols ]
    for sheet_name, _ in sheets.items():
        for header in range(0, max_header):
            try:
                df_try = pd.read_excel(str(path), sheet_name=sheet_name, engine=engine, header=header)
            except Exception:
                continue
            cols = _normalize_cols(df_try.columns)
            keys = [ _col_key(c) for c in cols ]
            # check if all expected keys exist in this header's columns (fuzzy: ignore case/spaces)
            if all(any(ek == k for k in keys) for ek in expected_keys):
                df_try.columns = cols
                logger.info(f'LOE detected sheet "{sheet_name}" header row {header}')
                return df_try
    return None

try:
    if not LOE_PATH.exists():
        logger.error(f'LOE file not found: {LOE_PATH}')
        raise FileNotFoundError(f'LOE file not found: {LOE_PATH}')

    # Define required columns (keep Date/DocNo optional but prefer present)
    required_loe_cols = [
        'Date','DocNo','Debit External','Credit External','PAN','Debit Amount','Entry Identifier'
    ]
    # Attempt smart detection across sheets/headers
    df_loe = detect_and_read_loe(LOE_PATH, required_loe_cols)
    if df_loe is None:
        # fallback: try sheet 1 with default header (original behaviour)
        df_loe = pd.read_excel(str(LOE_PATH), sheet_name=1, engine='openpyxl')
    df_loe.columns = _normalize_cols(df_loe.columns)
    logger.info(f'LOE file read: {LOE_PATH}')
    print("\nLoe file read...\nProceeding to outgoing... PLEASE WAIT")

    if not OUTGOING_PATH.exists():
        logger.error(f'Outgoing file not found: {OUTGOING_PATH}')
        raise FileNotFoundError(f'Outgoing file not found: {OUTGOING_PATH}')
    df_outgoing = pd.read_excel(str(OUTGOING_PATH), sheet_name=0, engine='openpyxl', header=1)
    df_outgoing.columns = _normalize_cols(df_outgoing.columns)
    logger.info(f'Outgoing file read: {OUTGOING_PATH}')
    print("\nOutgoing file read...\nProceeding to Analysis... PLEASE WAIT")

    # Basic schema validation (using normalized matching)
    required_out_cols = [
        'PAN','ORIGINALAMT','EXPTRANID','NETWORK','ORIGID'
    ]
    def _missing(df_cols, required):
        df_keys = set(_col_key(c) for c in df_cols)
        missing = [r for r in required if _col_key(r) not in df_keys]
        return missing

    try:
        validate_cols_missing = _missing(df_loe.columns, required_loe_cols)
        if validate_cols_missing:
            logger.error(f'Missing LOE columns: {validate_cols_missing}')
            raise KeyError(f'Missing LOE columns: {validate_cols_missing}')
    except Exception as e:
        logger.exception('LOE column validation failed')
        raise

    try:
        validate_out_missing = _missing(df_outgoing.columns, required_out_cols)
        if validate_out_missing:
            logger.error(f'Missing Outgoing columns: {validate_out_missing}')
            raise KeyError(f'Missing Outgoing columns: {validate_out_missing}')
    except Exception as e:
        logger.exception('Outgoing column validation failed')
        raise

except Exception as e:
    logger.exception('Error reading input files')
    print('Error reading input files:')
    print(e)
    sys.exit(1)

Processing Files started. This might take a minute... PLEASE WAIT.

Loe file read...
Proceeding to outgoing... PLEASE WAIT

Outgoing file read...
Proceeding to Analysis... PLEASE WAIT


In [21]:
# Data cleaning. Remove spaces on elements
df_loe.info()
# Strip whitespace from all string columns (avoid deprecated applymap)
for col in df_loe.select_dtypes(include=['object', 'string']).columns:
    df_loe[col] = df_loe[col].str.strip()
df_loe.columns = df_loe.columns.str.strip()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226218 entries, 0 to 226217
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Date                      226218 non-null  object 
 1   DocNo                     226218 non-null  int64  
 2   No                        226218 non-null  object 
 3   Debit External            226218 non-null  object 
 4   Debit Account             226218 non-null  object 
 5   Cur D                     226218 non-null  int64  
 6   Debit Amount              226218 non-null  float64
 7   Credit External           226218 non-null  object 
 8   Credit Account            226218 non-null  object 
 9   Cur C                     226218 non-null  int64  
 10  Credit Amount             226218 non-null  float64
 11  Entry Identifier          226218 non-null  object 
 12  PAN                       226218 non-null  object 
 13  Card product              226218 non-null  o

In [22]:
#DATA CLEANING FOR LOE

# ✅ Define target accounts
mc_acquiring_accounts = ["KES1403441530001", "KES1666600010001","USD1403441530001", "USD1666600010001"]


# ✅ Ensure columns are strings and strip spaces before filtering
df_loe["Debit External"] = df_loe["Debit External"].astype(str).str.strip()
df_loe["Credit External"] = df_loe["Credit External"].astype(str).str.strip()
df_loe["Entry Identifier"] = df_loe["Entry Identifier"].astype(str).str.strip()

# ✅ Filter to keep only rows where both Debit External & Credit External are in target_accounts
mc_acquiring = df_loe[
    df_loe["Debit External"].isin(mc_acquiring_accounts) & df_loe["Credit External"].isin(mc_acquiring_accounts)
].copy()



# ✅ Exclude rows where "Entry Identifier" is "ADJUSTMENT"
mc_acquiring = mc_acquiring[~mc_acquiring["Entry Identifier"].str.upper().eq("ADJUSTMENT")]


# ✅ Reset index (optional)
mc_acquiring = mc_acquiring.reset_index(drop=True)
# ✅ Display filtered results
mc_acquiring.to_excel(str(REPORT_DIR / 'filter_loe.xlsx'),index=False)
mc_acquiring.head()


,Date,DocNo,No,Debit External,Debit Account,Cur D,Debit Amount,Credit External,Credit Account,Cur C,Credit Amount,Entry Identifier,PAN,Card product,Additional Customer info
0,19/02/2025,671629962,1,USD1666600010001,61016012,840,300.0,USD1403441530001,USD1403441530001,840,300.0,OPEXT_P10,5156195493036544,,
1,19/02/2025,671631292,1,KES1403441530001,KES1403441530001,404,800.0,KES1666600010001,26924401,404,800.0,OPEXT_P10,5167678872651006,,
2,19/02/2025,671631314,1,KES1403441530001,KES1403441530001,404,400.0,KES1666600010001,26924401,404,400.0,OPEXT_P10,5586860000699114,,
3,19/02/2025,671631323,1,KES1403441530001,KES1403441530001,404,4960.0,KES1666600010001,01223203,404,4960.0,OPEXT_P10,5524860480025249,,
4,19/02/2025,671631337,1,KES1403441530001,KES1403441530001,404,1880.0,KES1666600010001,03359077,404,1880.0,OPEXT_P10,5477430008821086,,


In [23]:
#DATA CLEANING FOR OUTGOING

# ✅ Convert Data Types
df_outgoing["ORIGINALAMT"] = pd.to_numeric(df_outgoing["ORIGINALAMT"], errors="coerce").fillna(0.00).abs()
df_outgoing["ORIGTIME"] = pd.to_datetime(df_outgoing["ORIGTIME"], errors="coerce").fillna(pd.Timestamp("1970-01-01"))
df_outgoing["BUS. DAY"] = pd.to_datetime(df_outgoing["BUS. DAY"], errors="coerce").fillna(pd.Timestamp("1970-01-01"))

df_outgoing["EXPTRANID"] = df_outgoing["EXPTRANID"].astype(str).str[:25]  # Keep only first 25 characters

targ_network = [22]
targ_origid = ['KCBK']
targ_date = ['2025-02-19']
mc_acquiring_out = df_outgoing[
        df_outgoing["NETWORK"].isin(targ_network)& df_outgoing["ORIGID"].isin(targ_origid)#&df_outgoing["BUS. DAY"].astype(str).isin(targ_date)
].copy()



In [24]:
# ✅ Define Keys
mc_acquiring.loc[:,'key'] = mc_acquiring['PAN'].astype(str).str.strip() + mc_acquiring['Debit Amount'].astype(str)

mc_acquiring_out.loc[:,'key'] = mc_acquiring_out['PAN'].astype(str) + mc_acquiring_out['ORIGINALAMT'].astype(str)


In [25]:
#mc_acquiring.head()

In [26]:
mc_acquiring.loc[:, 'Status'] = mc_acquiring['key'].isin(mc_acquiring_out['key']).map({True: 'matched', False: 'Non-matched'})

print(mc_acquiring['Status'])

0       Non-matched
1           matched
2           matched
3           matched
4           matched
           ...     
9663        matched
9664        matched
9665        matched
9666        matched
9667        matched
Name: Status, Length: 9668, dtype: object


In [27]:
# ✅ Find duplicate keys in df_loe
duplicates_loe = mc_acquiring[mc_acquiring['key'].duplicated(keep=False)]  # Keep all occurrences

# ✅ Find duplicate keys in df_outgoing
duplicates_outgoing = mc_acquiring_out[mc_acquiring_out['key'].duplicated(keep=False)]  # Keep all occurrences

#duplicates_outgoing.to_excel('duplicates_out.xlsx', index=False, header=False)

In [28]:
report_loe = ['Debit External','Credit External','PAN','Status']
report_out = ['EXPTRANID','ID','EXPBATCHID','APPROVALCODE']
df_matching_report = pd.concat([mc_acquiring.loc[:, report_loe] , mc_acquiring_out.loc[:,report_out]],axis=1)

df_matching_report.to_excel(str(REPORT_DIR / 'Investigation_report.xlsx'),index=False)
# Export primary filtered datasets
mc_acquiring.to_excel(str(REPORT_DIR / 'filter_loe.xlsx'), index=False)
mc_acquiring_out.to_excel(str(REPORT_DIR / 'filter_outgoing.xlsx'), index=False)




In [29]:
# Export matched, all outgoing, and duplicate reports
mc_acquiring.to_excel(str(REPORT_DIR / 'matched_loe.xlsx'), index=False)
mc_acquiring_out.to_excel(str(REPORT_DIR / 'All_out.xlsx'), index=False)
# Save duplicates if any
try:
    duplicates_loe.to_excel(str(REPORT_DIR / 'duplicates_loe.xlsx'), index=False)
except NameError:
    pass
try:
    duplicates_outgoing.to_excel(str(REPORT_DIR / 'duplicates_out.xlsx'), index=False)
except NameError:
    pass
# Save analytics summary table if present
try:
    df.to_excel(str(REPORT_DIR / 'analytics_summary.xlsx'), index=False)
except NameError:
    pass
print('Exports written: matched_loe.xlsx, All_out.xlsx, Investigation_report.xlsx, filter_loe.xlsx, filter_outgoing.xlsx, analytics_summary.xlsx (when available)')

Exports written: matched_loe.xlsx, All_out.xlsx, Investigation_report.xlsx, filter_loe.xlsx, filter_outgoing.xlsx, analytics_summary.xlsx (when available)


In [30]:
## LOE analytics

usd_debit = ['USD1403441530001']
usd_credit = ['USD1666600010001']



loe_usd_sale = mc_acquiring[mc_acquiring['Debit External'].isin(usd_debit)]
loe_usd_rev = mc_acquiring[mc_acquiring['Debit External'].isin(usd_credit)]

total_usd_sale = loe_usd_sale['Debit Amount'].sum()
total_usd_rev = loe_usd_rev['Debit Amount'].sum()

total_usd = total_usd_sale - total_usd_rev
count_usd_sale=loe_usd_sale.shape[0]
count_usd_rev=loe_usd_rev.shape[0]

count_usd = count_usd_sale + count_usd_rev
print(f"USD Amount\t {total_usd} Count  {count_usd}")


kes_debit = ['KES1403441530001']
kes_credit = ['KES1666600010001']

loe_kes_sale = mc_acquiring[mc_acquiring['Debit External'].isin(kes_debit)]
loe_kes_rev = mc_acquiring[mc_acquiring['Debit External'].isin(kes_credit)]

total_kes_sale = loe_kes_sale['Debit Amount'].sum()
total_kes_rev = loe_kes_rev['Debit Amount'].sum()
count_kes_sale=loe_kes_sale.shape[0]
count_kes_rev=loe_kes_rev.shape[0]

count_kes = count_kes_sale + count_kes_rev

total_kes = total_kes_sale - total_kes_rev

grand_total = mc_acquiring['Debit External'].shape[0]
print(f"USD Amount\t {total_kes} \nTotal number of transactions  {grand_total }")


# OUTGOING analysis

sales_usd = [110,112]
rev_usd = [114]
usd=[840]
kes=[404]
sales_kes = [110,112,404]
rev_kes = [114,404]

mc_acquiring_out.loc[:,"Unique_ID"] = mc_acquiring_out["EXPTRANID"].astype(str).str[9:20]  # Extracts 00000008376
mc_acquiring_out.loc[:,"Sequence"] = mc_acquiring_out["EXPTRANID"].astype(str).str[20:]    # Extracts 00000

out_usd_sales = mc_acquiring_out[mc_acquiring_out['TRANCODE'].isin(sales_usd)&mc_acquiring_out['ORIGINALCCY'].isin(usd)]
out_usd_rev = mc_acquiring_out[mc_acquiring_out['TRANCODE'].isin(rev_usd)&mc_acquiring_out['ORIGINALCCY'].isin(usd)]

out_kes_sales = mc_acquiring_out[mc_acquiring_out['TRANCODE'].isin(sales_kes)&mc_acquiring_out['ORIGINALCCY'].isin(kes)]
out_kes_rev = mc_acquiring_out[mc_acquiring_out['TRANCODE'].isin(rev_kes)&mc_acquiring_out['ORIGINALCCY'].isin(kes)]

sum_usd_out = out_usd_sales['ORIGINALAMT'].sum()-out_usd_rev['ORIGINALAMT'].sum()
sum_kes_out = out_kes_sales['ORIGINALAMT'].sum() - out_kes_rev['ORIGINALAMT'].sum()

count_usd_out = out_usd_sales.shape[0]+out_usd_rev.shape[0]
count_kes_out = out_kes_sales.shape[0]+out_kes_rev.shape[0]
#
grand_total_out = count_usd_out + count_kes_out
print(sum_usd_out)



#differences

usd_diff= count_usd - count_usd_out
kes_diff = count_kes - count_kes_out

amount_usd = total_usd - sum_usd_out
amount_kes = total_kes - sum_kes_out
diff_total = grand_total - grand_total_out


# Creating structured data for the table
data = {
    "Reports": ["LOE", "","Total_", "Outgoing", "",'Total','Diffs Total',""],  # Merging LOE & Outgoing rows
    "Currency": ["USD", "KES","", "USD", "KES","",'USD','KES'],
    "Count": [count_usd, count_kes,grand_total, count_usd_out,count_kes_out,grand_total_out,usd_diff,kes_diff],
    "Amount": [total_usd, total_kes,'', sum_usd_out, sum_kes_out,"",amount_usd,amount_kes]
}

# Convert dictionary to pandas DataFrame
df = pd.DataFrame(data)

# Display the table in Jupyter Notebook
df


USD Amount	 525075.5399999999 Count  1236
USD Amount	 47280369.08 
Total number of transactions  9668
525925.5399999999


,Reports,Currency,Count,Amount
0,LOE,USD,1236,525075.54
1,,KES,8432,47280369.08
2,Total_,,9668,
3,Outgoing,USD,1234,525925.54
4,,KES,8431,47577575.08
5,Total,,9665,
6,Diffs Total,USD,2,-850.0
7,,KES,1,-297206.0


In [31]:
# Safe export helper: write DataFrame to REPORT_DIR with fallback timestamp on failure
from datetime import datetime
from pathlib import Path
import os

# use existing logger

def safe_save(df, fname):
    path = Path(fname)
    try:
        df.to_excel(path, index=False)
        logger.info(f'Saved {path}')
        return str(path)
    except PermissionError:
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        base = path.with_suffix('')
        ext = path.suffix
        alt = Path(f"{base}_{ts}{ext}")
        try:
            df.to_excel(alt, index=False)
            logger.info(f'Saved fallback {alt} due to PermissionError')
            return str(alt)
        except Exception as e:
            logger.exception(f'Failed to save fallback {alt}: {e}')
            return None
    except Exception as e:
        logger.exception(f'safe_save failed for {path}: {e}')
        return None

exports = {}
# Try saving main exports with fallback names (skip if variable not defined)
try:
    exports['filter_loe'] = safe_save(mc_acquiring, REPORT_DIR / 'filter_loe.xlsx')
except NameError:
    logger.warning('mc_acquiring not defined; skipping filter_loe export')
try:
    exports['filter_outgoing'] = safe_save(mc_acquiring_out, REPORT_DIR / 'filter_outgoing.xlsx')
except NameError:
    logger.warning('mc_acquiring_out not defined; skipping filter_outgoing export')
try:
    exports['matched_loe'] = safe_save(mc_acquiring, REPORT_DIR / 'matched_loe.xlsx')
except NameError:
    logger.warning('mc_acquiring not defined; skipping matched_loe export')
try:
    exports['All_out'] = safe_save(mc_acquiring_out, REPORT_DIR / 'All_out.xlsx')
except NameError:
    logger.warning('mc_acquiring_out not defined; skipping All_out export')
try:
    exports['investigation_report'] = safe_save(df_matching_report, REPORT_DIR / 'Investigation_report.xlsx')
except NameError:
    logger.warning('df_matching_report not defined; skipping investigation_report export')
try:
    exports['analytics_summary'] = safe_save(df, REPORT_DIR / 'analytics_summary.xlsx')
except NameError:
    logger.warning('df not defined; skipping analytics_summary export')
try:
    exports['duplicates_loe'] = safe_save(duplicates_loe, REPORT_DIR / 'duplicates_loe.xlsx')
except NameError:
    logger.warning('duplicates_loe not defined; skipping duplicates_loe export')
try:
    exports['duplicates_out'] = safe_save(duplicates_outgoing, REPORT_DIR / 'duplicates_out.xlsx')
except NameError:
    logger.warning('duplicates_outgoing not defined; skipping duplicates_out export')

logger.info('Export results:')
for k, v in exports.items():
    logger.info(f' {k}: {v}')

print('Export results:')
for k, v in exports.items():
    print(f' {k}: {v}')

Export results:
 filter_loe: E:\Automation\Card_recon\MC_\mc_reports\filter_loe.xlsx
 filter_outgoing: E:\Automation\Card_recon\MC_\mc_reports\filter_outgoing.xlsx
 matched_loe: E:\Automation\Card_recon\MC_\mc_reports\matched_loe.xlsx
 All_out: E:\Automation\Card_recon\MC_\mc_reports\All_out.xlsx
 investigation_report: E:\Automation\Card_recon\MC_\mc_reports\Investigation_report.xlsx
 analytics_summary: E:\Automation\Card_recon\MC_\mc_reports\analytics_summary.xlsx
 duplicates_loe: E:\Automation\Card_recon\MC_\mc_reports\duplicates_loe.xlsx
 duplicates_out: E:\Automation\Card_recon\MC_\mc_reports\duplicates_out.xlsx


In [32]:
#Visa issuing analysis

issuing_credit = ['KES1403500020001']

visa_issuing_credit = df_loe[df_loe['Credit External'].isin(issuing_credit)]

visa_issuing_debit = df_loe[
    df_loe['Debit External'].isin(issuing_credit) &
    df_loe['Credit External'].str.contains('KES165550', na=False)
]

print(f"Credit Amount. {visa_issuing_credit['Debit Amount'].sum():,.2f}")
print(f"Count.         {visa_issuing_credit['Debit Amount'].count():,.2f}")

print(f"Debit Amount.    -{visa_issuing_debit['Debit Amount'].sum():,.2f}")
print(f"Count.           -{visa_issuing_debit['Debit Amount'].count():,.2f}")

counts= visa_issuing_credit['Debit Amount'].count()+visa_issuing_debit['Debit Amount'].count()
issuing_usd = ['USD1403500020001']

issuing_value = visa_issuing_credit['Debit Amount'].sum() - visa_issuing_debit['Debit Amount'].sum()
print(f"Total Issuing. {issuing_value:,.2f}")
print(f"Total count.      {counts:,.2f}")


Credit Amount. 9,026,673.36
Count.         1,372.00
Debit Amount.    -92,648.82
Count.           -14.00
Total Issuing. 8,934,024.54
Total count.      1,386.00
